In [75]:
from math import log
import operator

In [76]:
def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing','flippers']
    #change to discrete values
    return dataSet, labels

In [77]:
def calcShannonEnt(dataSet):
    # 计算最后一列特征信息熵
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet: #the the number of unique elements and their occurance
#         print(featVec)
        currentLabel = featVec[-1]
#         print(currentLabel)
        if currentLabel not in labelCounts.keys(): 
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
#         print(labelCounts)
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob,2) #log base 2
    return shannonEnt

In [78]:
dataSet = [[1, 1, 'yes'],
            [1, 1, 'yes'],
            [1, 0, 'no'],
            [0, 1, 'no'],
            [0, 1, 'no']]
calcShannonEnt(dataSet)

0.9709505944546686

In [79]:
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]     #切开轴用于分裂
#             print(reducedFeatVec)
            reducedFeatVec.extend(featVec[axis+1:]) # reducedFeatVec添加分裂后部分
#             print(reducedFeatVec)
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [80]:
dataSet = [[1, 1, 'yes'],
            [1, 1, 'yes'],
            [1, 0, 'no'],
            [0, 1, 'no'],
            [0, 1, 'no']]
splitDataSet(dataSet = dataSet, axis = 1, value = 1)

[[1, 'yes'], [1, 'yes'], [0, 'no'], [0, 'no']]

In [81]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1      # 最后一列为标签,其余为特征
    baseEntropy = calcShannonEnt(dataSet) # 最后一列特征信息熵
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):        # 迭代所有特征
        featList = [example[i] for example in dataSet]# 创建这个特征列表
        uniqueVals = set(featList)       #得到特征列表唯一值
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)     
        infoGain = baseEntropy - newEntropy     #计算info gain; ie reduction in entropy
        if (infoGain > bestInfoGain):       #与已知最优gain对比
            bestInfoGain = infoGain         #存储对比后最优gain
            bestFeature = i
    return bestFeature                      #返回第i个特征的i值

In [82]:
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): 
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    # item遍历classCount
    # operator.itemgetter(1) 获取对象的第1个域的值
    return sortedClassCount[0][0]

In [83]:
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]

    if classList.count(classList[0]) == len(classList): 
        return classList[0]# 当剩下的所有的类都一样时停止分割
    if len(dataSet[0]) == 1: # 当数据集中没有更多特性时停止分割
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]       # 复制所有的标签，这样树就不会打乱已有的标签
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
    return myTree      

In [84]:
# 通过tree来判断分类
def classify(inputTree,featLabels,testVec):
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat = secondDict[key]
    if isinstance(valueOfFeat, dict): #  valueOfFeat是dict类型则为True
        classLabel = classify(valueOfFeat, featLabels, testVec)
    else: 
        classLabel = valueOfFeat
    return classLabel

In [86]:
dataset = [['F', '+10', '1'],
          ['F', '-10', '1'],
          ['F', '+10', '2'],
          ['F', '+10', '1'],
          ['M', '+10', '1'],
          ['M', '+10', '1'],
          ['M', '-10', '2']]
labels = ['bought', 'no bought']
mytree = createTree(dataset,labels)
classify(mytree,labels,['F', '+10', '1'])

ValueError: 'no bought' is not in list

In [87]:
mytree

{'no bought': {'+10': {'bought': {'M': '1', 'F': '1'}},
  '-10': {'bought': {'M': '2', 'F': '1'}}}}